In [13]:
import numpy as np
import math
import matplotlib.pyplot as plt
import QuantumTomography as qKLib
from QuantumTomography import TomoClass
import os
import glob
from pathlib import Path
import fnmatch
from optimization import Optimizer, function_fidelity, FidelityResults
from densitymatrix import DensityMatrix, apply_unitary_to_dm
from Kuwait_function import *

In [8]:
working_dir = os.getcwd()
working_dir_data = os.getcwd()+'\StateTomoData'

In [10]:
n_files=0
working_dir=r"C:\Users\nicol\Desktop\Experiment"
os.chdir(working_dir)

filenames = [i for i in glob.glob("StateTomography_2Layers_*")]
filenames.sort(key=os.path.getmtime)

index_to_file = {}

for index, filename in enumerate(filenames):
    os.chdir(f"{working_dir}\\{filename}")
    filenames_aux=[i for i in glob.glob("StateTomo_2Layers*")]
    for index_second, filenames_aux_second in enumerate(filenames_aux):
        index_to_file[n_files] = f"{filename}\\{filenames_aux_second}"
        n_files+=1
os.chdir(working_dir)

In [11]:
os.chdir(working_dir)
qubit_number=2

## Defining the columns of the data file we want to use as data to reconstruct the density matrix (eg.: HH HV VH and VV basis)
column_start_single_counts=0
column_stop_single_counts=4

column_start_coincidences=12
column_stop_coincidences=16


#####################################################################
#---------------------- STATE TOMOGRAPHY ----------------------------
#####################################################################

                
### Calculating the efficiencies of each detector and writing in the conf file
for index in range(len(index_to_file)):
    os.chdir(f"{working_dir}\\{index_to_file[index]}\\")
    datafiles=[i for i in glob.glob("*")]
    efficiencies=get_channels_eff(datafiles, qubit_number, column_start_coincidences, column_stop_coincidences, os.getcwd())
    f = open(r'C:\Users\nicol\Desktop\conf.txt','r')
    data_conf = f.readlines()
    f.close()
    data_conf[8] = str('conf['+'\'Efficiency\'] =')+'['+ str(efficiencies[0]) + ',' + str(efficiencies[1]) + ',' + str(efficiencies[2]) + ','+ str(efficiencies[3])+']'
    z = open(r'C:\Users\nicol\Desktop\conf.txt','w')
    print('Wrinting the Efficiency in the conf_file')
    z.writelines(data_conf)
    z.close()
    
    ### Opening the data files and writing the data in counts_aux array
    print('Getting the Counts')

    single_counts=set_raw_counts(datafiles, qubit_number, column_start_single_counts, column_stop_single_counts, os.getcwd())
    xp_counts_single=np.array(np.transpose(single_counts))
    
    coincidence_count = set_raw_counts(datafiles, qubit_number, column_start_coincidences, column_stop_coincidences, os.getcwd())
    xp_counts_coincidence=np.array(np.transpose(coincidence_count))
    
    ### Writing the data in the right order
    #D,A,R,L,H,V
    print('Ordering the counts')

    List_Bases = get_char_iterator(qubit_number)
    Projector_basis = np.reshape(np.array(get_proj_iterator(qubit_number)),(36,4))
    
    j = 0
    tomo_input = 1j * np.zeros((len(xp_counts_single)*2**qubit_number,2 ** qubit_number + 4 * qubit_number+1 ))
    intensity = np.zeros((36))
    
    for i in range(len(xp_counts_single)):
        for w in range(len(single_counts)):
            tomo_input[j,0] = 1
            tomo_input[j,2*qubit_number+1+w] = xp_counts_coincidence[i,w]
            tomo_input[j,2 ** qubit_number + 2 * qubit_number + 1+0] = Projector_basis[j,0]
            tomo_input[j,2 ** qubit_number + 2 * qubit_number + 1+1] = Projector_basis[j,1]
            tomo_input[j,2 ** qubit_number + 2 * qubit_number + 1+2] = Projector_basis[j,2]
            tomo_input[j,2 ** qubit_number + 2 * qubit_number + 1+3] = Projector_basis[j,3]
            intensity[j] = 1
            j=j+1
            ### Writing the projector in the tomo_input
tomography = 1j* np.zeros((9,13))
tomography_r = 1j* np.zeros((9,13))

for i in range(len(xp_counts_single)):
    tomography[i] = tomo_input[i*4]
            ### Wrinting the data in the right dimension
tomography_r[0] = tomography[8]
tomography_r[1] = tomography[6]
tomography_r[2] = tomography[7]
tomography_r[3] = tomography[2]
tomography_r[4] = tomography[0]
tomography_r[5] = tomography[1]
tomography_r[6] = tomography[5]
tomography_r[7] = tomography[3]
tomography_r[8] = tomography[4]

print(tomography_r)








Wrinting the Efficiency in the conf_file
Getting the Counts
['xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz']
['xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz']
Ordering the counts
[[1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         3.54087000e+05+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         1.00000000e+00+0.j
  0.00000000e+00+0.j         1.00000000e+00+0.j
  0.00000000e+00+0.j        ]
 [1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         1.09550000e+05+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         1.00000000e+00+0.j
  0.00000000e+00+0.j         7.07106781e-01+0.j
  7.07106781e-01+0.j        ]
 [1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         2.21461000e+05+0.j
  0.00000000e+00+0.j         0

In [16]:
t = qKLib.Tomography()
t.importConf(r'C:\Users\nicol\Desktop\conf.txt')
[rho, intensity, fval] = t.state_tomography(tomography_r)
qKLib.printLastOutput(t)


State: 
0.499            0.0554-i0.0821   0.31-i0.302      2.24e-03-i0.229  
0.0554+i0.0821   0.0196           0.084+i0.0174    0.0378-i0.025    
0.31+i0.302      0.084-i0.0174    0.375            0.14-i0.14       
2.24e-03+i0.229  0.0378+i0.025    0.14+i0.14       0.105            
intensity : 7.00e+05 +/- 149.0
fval : 2.20e+06 +/- 463.0
concurrence : 0.291 +/- 2.39e-05
tangle : 0.085 +/- 1.39e-05
entropy : 1.34e-06 +/- 1.74e-07
linear_entropy : 1.04e-07 +/- 1.41e-08
negativity : 0.291 +/- 2.39e-05
purity : 0.999 +/- 1.41e-08


In [17]:
print(rho)
print(tomo_input)   
print(efficiencies)
print(xp_counts_coincidence*efficiencies)


[[0.4999701 -7.73459471e-19j 0.05545739-8.21091400e-02j
  0.3101875 -3.02357935e-01j 0.00224027-2.29218050e-01j]
 [0.05545739+8.21091400e-02j 0.01963604-4.89063443e-19j
  0.08406211+1.74035350e-02j 0.03789254-2.50572742e-02j]
 [0.3101875 +3.02357935e-01j 0.08406211-1.74035350e-02j
  0.3752957 +1.37649821e-18j 0.14000997-1.40854854e-01j]
 [0.00224027+2.29218050e-01j 0.03789254+2.50572742e-02j
  0.14000997+1.40854854e-01j 0.10509815-1.13975299e-19j]]
[[ 1.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          1.81843000e+05+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          7.07106781e-01+0.j
   7.07106781e-01+0.j          7.07106781e-01+0.j
   7.07106781e-01+0.j        ]
 [ 1.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   5.56818000e+05+0.j          0.00000000e+00+0.j
   0.00000000e+0

In [18]:
print(tomography)
    

[[1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         1.81843000e+05+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         7.07106781e-01+0.j
  7.07106781e-01+0.j         7.07106781e-01+0.j
  7.07106781e-01+0.j        ]
 [1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         6.91983000e+05+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         7.07106781e-01+0.j
  7.07106781e-01+0.j         7.07106781e-01+0.j
  0.00000000e+00+0.70710678j]
 [1.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         7.71343000e+05+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         7.07106781e-01+0.j
  7.07106781e-01+0.j         1.00000000e+00+0.j
  0.00000000e+00+0.j        ]
 [1.00000000e+00+0.j         0.00000000e+00+0.

In [24]:
##########################################################
#----- OPTIMIZATION OF MAX FIDELITY UP TO UNITARIES ------
##########################################################
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))

states=DensityMatrix(rho)
fid=np.zeros((n_files))
optimized_matrix=np.zeros((n_files,2**qubit_number,2**qubit_number), dtype='complex')

guess=np.array([0, 0, 0])
bounds=[(-np.pi,np.pi)]*3

opt=Optimizer(guess, function_fidelity, results=FidelityResults)

result=opt.optimize(states, bell, bounds=bounds)
results=result

In [25]:
target=bell
U=results.u
target_ini=np.transpose(np.conjugate(U))@bellmatrix@U

print(np.round(states.fidelity(target_ini),5))

print("Fidelity: " + str(qKLib.fidelity(np.transpose(np.conjugate(U))@rho@U,target)))

(0.64579+0j)
Fidelity: 0.6411902354688339


In [ ]:
qKLib.makeRhoImages(rho, plt, True)
plt.show()


In [ ]:
print(intensity)
print(np.shape(intensity))
intensity2 = np.array([1, 1, 1, 1, 1, 1])
print(intensity2)
print(np.shape(intensity2))

In [ ]:
print(xp_counts_coincidence)



In [14]:
state= []
state=getTomoInputTemplate(4,2)


NameError: name 'getTomoInputTemplate' is not defined